In [16]:
"""
fetch_prices.py

Fetches daily stock price data (non-adjusted) from Alpha Vantage and stores it in PostgreSQL.
"""

import os
import requests
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
from models.price_data_model import Base, PriceData

# Load environment variables
load_dotenv()

True

In [18]:


# Configuration
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'
DATABASE_URL = os.getenv("DATABASE_URL")

# SQLAlchemy setup
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def fetch_daily_prices(ticker):
    """Fetch daily stock prices (not adjusted) from Alpha Vantage and store in DB."""
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': ticker,
        'outputsize': 'full',
        'apikey': API_KEY
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code}")

    json_data = response.json()

    # Debug: Check if API returned valid data
    if 'Time Series (Daily)' not in json_data:
        print("API Error Response:", json_data)
        raise ValueError(f"No data returned for ticker {ticker}")

    data = json_data['Time Series (Daily)']
    records = []

    for date_str, values in data.items():
        record = PriceData(
            ticker=ticker,
            date=datetime.strptime(date_str, '%Y-%m-%d').date(),
            open=float(values['1. open']),
            high=float(values['2. high']),
            low=float(values['3. low']),
            close=float(values['4. close']),
            adjusted_close=None,  # Not available in this endpoint
            volume=int(values['5. volume']),
            source_api='Alpha Vantage',
            interval='1d',
            currency='USD',
            data_type='historical',
            created_at=datetime.utcnow()
        )
        records.append(record)

    session.bulk_save_objects(records)
    session.commit()
    print(f"Inserted {len(records)} price records for {ticker}")

if __name__ == '__main__':
    Base.metadata.create_all(engine)
    fetch_daily_prices('NVDA')  # Use demo-compatible ticker


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "price_data_pkey"
DETAIL:  Key (ticker, date, "interval", source_api)=(NVDA, 2025-06-10, 1d, Alpha Vantage) already exists.

[SQL: INSERT INTO price_data (ticker, date, interval, source_api, open, high, low, close, volume, currency, data_type, created_at) VALUES (%(ticker__0)s, %(date__0)s, %(interval__0)s, %(source_api__0)s, %(open__0)s, %(high__0)s, %(low__0)s, %(close__0)s, % ... 209460 characters truncated ... _999)s, %(close__999)s, %(volume__999)s, %(currency__999)s, %(data_type__999)s, %(created_at__999)s)]
[parameters: {'date__0': datetime.date(2025, 6, 11), 'open__0': 144.61, 'volume__0': 167694044, 'interval__0': '1d', 'source_api__0': 'Alpha Vantage', 'created_at__0': datetime.datetime(2025, 6, 12, 10, 33, 35, 858072), 'currency__0': 'USD', 'data_type__0': 'historical', 'ticker__0': 'NVDA', 'low__0': 141.87, 'high__0': 144.99, 'close__0': 142.83, 'date__1': datetime.date(2025, 6,